In [25]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [26]:
mail = pd.read_csv('/content/mail_data.csv')

In [27]:
mail.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
mail['Category'].value_counts()


Category
ham     4825
spam     747
Name: count, dtype: int64

In [29]:
mail.isnull().sum()

Category    0
Message     0
dtype: int64

In [30]:
#replacing null values (if exist)
mail_data = mail.where((pd.notnull(mail)),'')

In [31]:
mail_data.shape

(5572, 2)

LABEL ENCODING

In [32]:
encoder = LabelEncoder()

In [33]:
labels =encoder.fit_transform(mail_data.Category)
mail_data['target']=labels

In [34]:
mail_data.head()

,Category,Message,target
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


0 --> ham  
1 --> spam

In [35]:
#user defined label encoding
mail_data.loc[mail_data['Category']== 'spam','Category',] = 0
mail_data.loc[mail_data['Category']== 'ham','Category',] = 1

In [36]:
mail_data.head()


,Category,Message,target
0,1,"Go until jurong point, crazy.. Available only ...",0
1,1,Ok lar... Joking wif u oni...,0
2,0,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,1,U dun say so early hor... U c already then say...,0
4,1,"Nah I don't think he goes to usf, he lives aro...",0


In [37]:
mail_data = mail_data.drop(columns = 'target',axis=1)

In [38]:
#seperating text and label
x = mail_data['Message']
y = mail_data['Category']

In [39]:
print(x,y)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object 0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


Train test data

In [40]:
x_train, x_test , y_train, y_test = train_test_split(x,y,stratify=y,test_size=0.2,random_state=3)

In [41]:
#transform text to feature that can be used by logisticRegression
feature_extraction= TfidfVectorizer(min_df=1, stop_words='english',lowercase= True)
x_train_feat = feature_extraction.fit_transform(x_train)
x_test_feat = feature_extraction.transform(x_test)

In [42]:
print(x_train_feat)

  (0, 0)	0.23628394623676158
  (0, 1657)	0.28101404009316056
  (0, 6468)	0.26793132631329497
  (0, 4557)	0.28101404009316056
  (0, 421)	0.25144905621529934
  (0, 4306)	0.26793132631329497
  (0, 5029)	0.17467075796896542
  (0, 2644)	0.28101404009316056
  (0, 1540)	0.17407870571957915
  (0, 6330)	0.24059246244542992
  (0, 3627)	0.25144905621529934
  (0, 3113)	0.28101404009316056
  (0, 1193)	0.22908400928709988
  (0, 1857)	0.17073786814794129
  (0, 3806)	0.28101404009316056
  (0, 2353)	0.28101404009316056
  (1, 4076)	0.1543395674723974
  (1, 5416)	0.28967873139399253
  (1, 6977)	0.1293522168838017
  (1, 765)	0.21147006367289747
  (1, 1068)	0.1771111381363262
  (1, 3089)	0.13752009582621935
  (1, 3961)	0.20073435617244362
  (1, 3828)	0.13684128003316173
  (1, 2113)	0.19851614641109666
  :	:
  (4456, 3913)	0.23883125341667502
  (4456, 2138)	0.23883125341667502
  (4456, 6568)	0.23883125341667502
  (4456, 6646)	0.22771237505351186
  (4456, 5447)	0.2198234053076842
  (4456, 4878)	0.21370424497

In [43]:
#convert y_train y_test as integer
y_train = y_train.astype('int')
y_test = y_test.astype('int')

Training the model

In [44]:
model = LogisticRegression()
model.fit(x_train_feat,y_train)

LogisticRegression()

Evaluating the trained model

In [46]:
#evaluating training prediction
training_prediction = model.predict(x_train_feat)
accuracy= accuracy_score(training_prediction,y_train)
print(accuracy)

0.9670181736594121


In [47]:
#evaluating testing prediction
testing_prediction = model.predict(x_test_feat)
accuracy= accuracy_score(testing_prediction,y_test)
print(accuracy)

0.9721973094170404


In [50]:
# classifying whether an input mail is spam or ham
input_mail=["Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."]
input_feat_mail = feature_extraction.transform(input_mail)
result= model.predict(input_feat_mail)
if result==0:
  print("Mail is a spam")
else:
  print("Mail is a ham")

Mail is a ham
